In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas


In [3]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('https://raw.githubusercontent.com/sambasnet/Mental-Health_Dashboard/main/survey_mental_health.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [4]:
df

,Timestamp,Age,Gender,Gender_clean,Country,state,self_employed,family_history,Received_treatment,work_interfere,...,leave,mental_health_consequence,phys_health_consequence,coworkers_are_aware,supervisor_is_aware,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,27/08/2014 11:29,37,Female,Female,United States,IL,NaN,No,Yes,Often,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,27/08/2014 11:29,44,M,Male,United States,IN,NaN,No,No,Rarely,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,27/08/2014 11:29,32,Male,Male,Canada,NaN,NaN,No,No,Rarely,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,27/08/2014 11:29,31,Male,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,27/08/2014 11:30,31,Male,Male,United States,TX,NaN,No,No,Never,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,12/09/2015 11:17,26,male,Male,United Kingdom,NaN,No,No,Yes,NaN,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,26/09/2015 01:07,32,Male,Male,United States,IL,No,Yes,Yes,Often,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,07/11/2015 12:36,34,male,Male,United States,CA,No,Yes,Yes,Sometimes,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,30/11/2015 21:25,46,f,Female,United States,NC,No,No,No,NaN,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


In [5]:
df.columns

Index(['Timestamp', 'Age', 'Gender', 'Gender_clean', 'Country', 'state',
       'self_employed', 'family_history', 'Received_treatment',
       'work_interfere', 'no_employees', 'remote_work', 'tech_company',
       'benefits_provided', 'care_options', 'wellness_program', 'seek_help',
       'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers_are_aware', 'supervisor_is_aware',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [19]:
df['Received_treatment']

0       Yes
1        No
2        No
3       Yes
4        No
       ... 
1254    Yes
1255    Yes
1256    Yes
1257     No
1258    Yes
Name: Received_treatment, Length: 1259, dtype: object

In [20]:
treatment_by_gender = df.groupby('Gender_clean')['Received_treatment'].value_counts().unstack().fillna(0)

In [21]:
treatment_by_gender

Received_treatment,No,Yes
Gender_clean,,
Female,77.0,172.0
Male,542.0,452.0
Missing,0.0,5.0
Queer,2.0,5.0
Transgender,1.0,3.0


In [22]:
idf = df.interactive()

In [38]:
yaxis_health = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['mental_health_consequence', 'mental_health_interview','Received_treatment'],
    button_type='success'
)

In [39]:
health_pipeline =idf.groupby(['Gender_clean'])[yaxis_health].value_counts().unstack().fillna(0)

In [54]:
health_plot = health_pipeline.hvplot(
    x='Gender_clean',
    stacked=True,
    title='Count of Receive Treatment per Gender',
    xlabel='Gender',
    ylabel='Count'
)

In [57]:
health_plot

In [58]:
health_table = health_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')

In [59]:
health_table

In [76]:
health_plot_bar =health_pipeline.hvplot.bar(
    x='Gender_clean',
    stacked=True,
    title='Count of Receive Treatment per Gender',
    xlabel='Gender',
    ylabel='Count'
)

In [77]:
health_plot_bar

In [78]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World Mental-health dashboard', 
    sidebar=[pn.pane.Markdown("# World Mental-Health Analytics"), 
             pn.pane.Markdown("#### World Mental Health Day is observed on October 10 every year to raise awareness about mental health issues and promote efforts to support mental health. It was first observed in 1992 at the initiative of the World Federation for Mental Health. The day provides an opportunity to address the stigma surrounding mental health and encourage individuals to seek help for mental health issues. It also serves as a reminder to governments, policymakers, and healthcare providers to prioritize mental health and work towards providing better mental health services and support to those in need."), 
             pn.pane.PNG('world_day.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings")   
             ],
    main=[pn.Row(pn.Column(yaxis_health, 
                           health_plot.panel(width=700), margin=(0,25)), 
                 health_table.panel(width=500)), 
          pn.Row(pn.Column(yaxis_health, health_plot_bar.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();